<a href="https://colab.research.google.com/github/ptleskin/Integer-Combinations-by-Rolling-a-Dice/blob/main/Integer_Combinations_by_Rolling_a_Dice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and imports

In [ ]:
!pip install sympy

In [ ]:
from collections import Counter, defaultdict
from itertools import product
import sympy as sp
from sympy.abc import f, x, k

In [ ]:
def printCoeffs(f) -> None:
  lst = sp.Poly(f).all_coeffs()
  for i, c in enumerate(lst, start=1):
    if c:
      print(f'{len(lst)-i} can be achieved in {c} {"different ways" if c>1 else "way"}')

# Integer Combinations by Rolling a Dice

The idea for the problem solving in this colab come from a question in a Facebook post:

*Using a standard six-sided die, it is possible to obtain a specific sum by rolling the die one or more times. For instance, the sum of 3 can be achieved in four ways: $1+1+1$, $1+2$, $2+1$, and $3$. How many different ways are there to obtain the sum of $7$?*


## Solution by brute force simulation

In [ ]:
# numbers as on the faces of a dice:
pips = [1, 2, 3, 4, 5, 6]

'''
Simulate throwing 1 to 10 dices, calculate the sum of pips and increase that value in dictionary cn
'''
cn = defaultdict(int)

for number_of_dice in range(1,11):
  for k in product(pips, repeat=number_of_dice):
    cn[sum(k)] += 1

for k,v in cn.items():
  if k<11:
    print(f'Total sum of {k} can achieved in {v} {"different ways" if k>1 else "way"}.')

Total sum of 1 can achieved in 1 way.
Total sum of 2 can achieved in 2 different ways.
Total sum of 3 can achieved in 4 different ways.
Total sum of 4 can achieved in 8 different ways.
Total sum of 5 can achieved in 16 different ways.
Total sum of 6 can achieved in 32 different ways.
Total sum of 7 can achieved in 63 different ways.
Total sum of 8 can achieved in 125 different ways.
Total sum of 9 can achieved in 248 different ways.
Total sum of 10 can achieved in 492 different ways.


## Solution by using a generating polynomial

Generating function for throwing one dice with resulting values {1, 2, 3, 4, 5, 6} is given by the polynomial

$f(x) = 1 \cdot x + 1 \cdot x^2 + 1 \cdot x^3 + 1 \cdot x^4 + 1 \cdot x^5 + 1 \cdot x^6 $

The coefficients for all powers of $x$ is $1$ so there is only one way to achieve each value .

Furthermore, the possible output of throwing two dice by

$f(x)^2 = 1 \cdot x^2 + 2 \cdot x^3 + 3 \cdot x^4 + 4 \cdot x^5 + 5 \cdot x^6 + 6 \cdot x^7 + 5 \cdot x^8 + 4 \cdot x^9 + 3 \cdot x^{10} + 2 \cdot x^{11} + 1 \cdot x^{12}$

meaning that, e.g., value of $4$ can be achieved in 3 different ways $(1+3, 2+2, 3+1)$ or likewise number $7$ in $6$ different ways.



In [ ]:
'''
generating function for throwing one dice
'''
f = x**6 + x**5 + x**4 + x**3 + x**2 + x

printCoeffs(f)

6 can be achieved in 1 way
5 can be achieved in 1 way
4 can be achieved in 1 way
3 can be achieved in 1 way
2 can be achieved in 1 way
1 can be achieved in 1 way


In [ ]:
'''
generating function for throwing a dice for 2 times:
'''
printCoeffs(f**2)

12 can be achieved in 1 way
11 can be achieved in 2 different ways
10 can be achieved in 3 different ways
9 can be achieved in 4 different ways
8 can be achieved in 5 different ways
7 can be achieved in 6 different ways
6 can be achieved in 5 different ways
5 can be achieved in 4 different ways
4 can be achieved in 3 different ways
3 can be achieved in 2 different ways
2 can be achieved in 1 way


In [ ]:
'''
generating function for throwing a dice for 1, 2, or 3 times:
'''
printCoeffs(f + f**2 + f**3)

18 can be achieved in 1 way
17 can be achieved in 3 different ways
16 can be achieved in 6 different ways
15 can be achieved in 10 different ways
14 can be achieved in 15 different ways
13 can be achieved in 21 different ways
12 can be achieved in 26 different ways
11 can be achieved in 29 different ways
10 can be achieved in 30 different ways
9 can be achieved in 29 different ways
8 can be achieved in 26 different ways
7 can be achieved in 21 different ways
6 can be achieved in 16 different ways
5 can be achieved in 11 different ways
4 can be achieved in 7 different ways
3 can be achieved in 4 different ways
2 can be achieved in 2 different ways
1 can be achieved in 1 way


### Throwing dice for arbitrary times

Calculating the number of all possible values of throwing a dice an arbitrary $N \in [1, \infty]$ times is given by the sum of the geometric series:

$\sum_{n = 1}^{\infty} f(x)^n = \frac{f(x)}{1-f(x)}$

In [ ]:
sp.series(f/(1-f), x, 0, 20)

x + 2*x**2 + 4*x**3 + 8*x**4 + 16*x**5 + 32*x**6 + 63*x**7 + 125*x**8 + 248*x**9 + 492*x**10 + 976*x**11 + 1936*x**12 + 3840*x**13 + 7617*x**14 + 15109*x**15 + 29970*x**16 + 59448*x**17 + 117920*x**18 + 233904*x**19 + O(x**20)

So, looking at the coefficient of the term $8x^4$ reveals that the result 4 can be achieved in the following 8 different ways:
$(1+1+1+1, 1+1+2, 1+2+1, 2+1+1, 3+1, 2+2, 1+3, 4)$

So the generating function has coefficients $[1,2,4,8,16,32,63,126,\dots]$ but why does this series diverge from the powers of two when $n \geq 7$?

To study that let's have a look at the case by a polynomial long division. First we will write the geometric sum as a rational function:

$f(x) = x^6 + x^5 + x^4 +x^3 + x^2 + x = \frac{x^7-x}{x-1}$

$\sum_{n = 1}^{\infty} f(x)^n = \frac{f(x)}{1-f(x)} = \frac{\frac{x^7-x}{x-1}}{1-\frac{x^7-x}{x-1}} = \frac{x^7-x}{x-1-(x^7-x)} = \frac{x-x^7}{1-2x+x^7}$

By applying a polynomial long division we will get:

<img width="600px" src="https://raw.githubusercontent.com/ptleskin/Integer-Combinations-by-Rolling-a-Dice/main/images/long_division.png">

So the term of highest degrees $x^7$ causes the series to diverge from the powers of two after $n=7$.

Generally, the plain generating function of $\frac{1}{1-2x}$ would generate the powers of two:

In [ ]:
sp.series(1/(1-2*x), x, 0, 10)

1 + 2*x + 4*x**2 + 8*x**3 + 16*x**4 + 32*x**5 + 64*x**6 + 128*x**7 + 256*x**8 + 512*x**9 + O(x**10)